In [22]:
from pyhive import hive
import pandas as pd
import numpy as np
import re
import nltk
import string
import datetime
import json
from datetime import date

In [23]:
#today = date.today().strftime("%Y-%m-%d")
today = '2020-05-13'
conn = hive.Connection(host='localhost',database='default', auth='NOSASL')


In [24]:
# query the table to a new dataframe
df = pd.read_sql("SELECT country, confirm, death, cured FROM covid_main where pdate ='" + today +"'", conn)
df.head()

,country,confirm,death,cured
0,Worldwide,4278180.0,292376.0,1502620.0
1,United States,1400741.0,83082.0,237256.0
2,Russia,242271.0,2212.0,48003.0
3,Spain,228691.0,27104.0,140823.0
4,United Kingdom,226463.0,32692.0,NaN


In [25]:
# check if any null value
df.isnull().sum()

country     0
confirm    37
death      40
cured      53
dtype: int64

In [26]:
# replace the null value with 0
df['confirm'] = df['confirm'].fillna(0).astype(int)
df['death'] = df['death'].fillna(0).astype(int)
df['cured'] = df['cured'].fillna(0).astype(int)

In [27]:
# check if null value still exist
df.isnull().sum()

country    0
confirm    0
death      0
cured      0
dtype: int64

In [28]:
# conn = hive.Connection(host='localhost',database='default', auth='NOSASL')
# query the table to a new dataframe
dfT = pd.read_sql("SELECT line FROM covid_twitter where pdate = '" + today +"'", conn)
dfT.head()

,line
0,"{""created_at"": ""Wed May 13 12:36:03 +0000 202..."
1,"{""created_at"": ""Wed May 13 12:36:03 +0000 202..."
2,"{""created_at"": ""Wed May 13 12:36:03 +0000 202..."
3,"{""created_at"": ""Wed May 13 12:36:03 +0000 202..."
4,"{""created_at"": ""Wed May 13 12:36:03 +0000 202..."


In [29]:
tweets = []
for line in dfT['line']:
    tweets.append(json.loads(line))

#columns of the csv file
COLS = ['id', 'created_at', 'text']

RDB = {}
for var in COLS:
    Text = []
    for i in range(0,len(tweets)):
        Text.append(tweets[i][var])
    RDB[var]=Text

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

##Cleaning & Convert Twitter Data to dataframe
def TwitterCleaning(DB):
    
    HDB = pd.DataFrame(DB)
    
    #Converting Datetime
    HDB['created_at'] = pd.to_datetime(HDB['created_at'])

    #Cleaning HTML Tags
    HDB['text']=HDB['text'].apply(lambda x: re.sub('<[^>]*>','',x))

    #Cleaning URL link
    HDB['text']=HDB['text'].apply(lambda x: re.sub(r"http\S+", "", x))

    #Cleaning punctuations
    HDB['text']=HDB['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

    #lower Case
    HDB['text']=HDB['text'].apply(lambda x: x.lower())
    
    #Remove Emoji
    HDB['text']=HDB['text'].apply(lambda x: emoji_pattern.sub(r'',x))
   
    return HDB

df2 = TwitterCleaning(RDB)
df2.head()

,id,created_at,text
0,1260549368094089216,2020-05-13 12:36:03+00:00,coronavirus covid19 and the rise of remote rec...
1,1260549367888515072,2020-05-13 12:36:03+00:00,no referral from a medical professional or hea...
2,1260549367410253826,2020-05-13 12:36:03+00:00,the most outrageously awesome ways tech compan...
3,1260549366470914048,2020-05-13 12:36:03+00:00,he warned of the covid19 pandemic early on — n...
4,1260549365577457667,2020-05-13 12:36:03+00:00,joanamamombe as an honourable member of parlia...
